In [1]:
import datetime
import pandas as pd

In [2]:
holidays = {
    "thanks": {
        "days": [datetime.datetime(year=2019, month=11, day=28) + datetime.timedelta(days=x) for x in range(4)],
        "senior": ['senior41', 'senior41', 'senior41', 'senior41'],
        "junior": ['junior21', 'junior21', 'junior22', 'junior22']
    },
    #"july4": None,
    #"memorial": None,
    #"labor": None
}


In [3]:
residents = [
    "senior51",
    "senior52",
    "senior41",
    "senior42",
    "junior31",
    "junior32",
    "junior21",
    "junior22",
]
seniors = [x for x in residents if x.find('senior') > -1]
juniors = [x for x in residents if x.find('junior') > -1]

In [4]:
def rand_from_l(l):
    """
    return a random element from a list
    """
    index = int(random.random() * len(l))
    return l[index]
def is_fri_sat_sun(day):
    return day.weekday() in (4,5,6)
def is_fri_sun(day):
    return day.weekday() in (4,6)
def mon_thru_fri(day):
    return day.weekday() in (0,1,2,3,4)

def count_totals(calls):
    names = {}
    for elem in calls:
        senior = elem[1]
        junior = elem[2]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

def count_weekends(calls):
    names = {}
    for elem in calls:
        day_of_week = elem[0].weekday()
        if day_of_week not in (5,6):
            continue
        senior = elem[1]
        junior = elem[2]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

def score_it(calls):
    totals = count_totals(calls)
    values = list(totals.values())
    p1 = max(values) - min(values)
    
    totals = count_weekends(calls)
    values = list(totals.values())
    p2 = max(values) - min(values)
    return p1 + p2

def get_best_schedule(n_bootstraps=1000):
    best_call = random_call_schedule()
    for i in range(n_bootstraps):
        my_call = random_call_schedule()
        my_score = score_it(my_call)
        if score_it(my_call) < score_it(best_call):
            best_call = my_call
    return best_call

In [5]:
import random

def is_holiday(day):
    for k,v in holidays.items():
        if day in v['days']:
            my_index = v['days'].index(day)
            return v['junior'][my_index], v['senior'][my_index], 
    return None

def select_mon_thru_fri(calls, call_junior, call_senior):
    if len(calls) == 0:
        return None, None
    last_senior = calls[-1][1]
    last_junior = calls[-1][2]
    while call_junior == last_junior:
        call_junior = rand_from_l(juniors)
    while call_senior == last_senior:
        call_senior = rand_from_l(seniors)
    return call_junior, call_senior


def random_call_schedule(start_day=datetime.datetime(2019, 7, 1, 0, 0), 
                         end_day = datetime.datetime(2020, 1, 1, 0, 0)):
    weekend_senior = seniors[0]
    fri_sun_junior = None
    calls = []
    day = start_day
    day_num = -1
    while day < end_day:
        day_num += 1
        call_senior = rand_from_l(seniors)
        call_junior = rand_from_l(juniors)
        day = start_day + datetime.timedelta(days=day_num)
        
        retval = is_holiday(day)
        if retval is not None:
            call_junior, call_senior = retval
            calls.append([day, call_senior, call_junior])
            continue
            
        if mon_thru_fri(day):
            r1, r2 = select_mon_thru_fri(calls, call_junior, call_senior)
            if r1 is not None:
                call_junior, call_senior = r1, r2
        
        if day.weekday() == 4:  # Friday set the junior and senior for the weekend
            weekend_senior = call_senior
            fri_sun_junior = call_junior
            
        if is_fri_sat_sun(day):
            call_senior = weekend_senior
        if is_fri_sun(day):
            call_junior = fri_sun_junior
        while day.weekday() == 5 and call_junior == fri_sun_junior:  # Don't let a junior do fri and sat
            call_junior = rand_from_l(juniors)

        calls.append([day, call_senior, call_junior])
    return calls

In [6]:
import time
start_time = datetime.datetime.now()
best_call = get_best_schedule(1000)
end_time = datetime.datetime.now()
print(end_time - start_time)

0:00:00.815572


In [9]:
"""
Only the first two headers in this list are required, the rest are optional.

Important: The headers must be in English as shown below. 
If any event details have commas (like the location example below), 
you can include them by using quotation marks around the text

Subject
The name of the event, required.
Example: Final exam

Start Date
The first day of the event, required.
Example: 05/30/2020
"""
import pandas as pd
def date_to_s(d):
    return "%s/%s/%s" % (d.month, d.day, d.year)


def to_gcal(calls, fname):
    table = []
    
    for c in calls:
        my_day = date_to_s(c[0])
        table.append([c[1], my_day])
        table.append([c[2], my_day])
    df = pd.DataFrame(table, columns=["Subject","Start Date"])
    df.to_csv(fname, index=None)
    return df

In [10]:
df = to_gcal(best_call, "call.csv")

In [11]:
count_totals(best_call)

{'senior52': 45,
 'junior31': 49,
 'senior51': 45,
 'junior21': 45,
 'senior42': 50,
 'junior32': 47,
 'senior41': 45,
 'junior22': 44}

In [12]:
count_weekends(best_call)

{'senior41': 12,
 'junior31': 14,
 'junior22': 14,
 'senior51': 12,
 'senior52': 14,
 'junior21': 12,
 'junior32': 12,
 'senior42': 14}